<a href="https://colab.research.google.com/github/giyushino/clip-vit-large-patch14-batch/blob/main/drawing_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install datasets
from transformers import CLIPProcessor, CLIPModel
import torch
from datasets import load_dataset
import torch.optim as optim
import torch.nn.functional as F
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
import numpy as np
import torch
import time
from PIL import Image

datasets2 = []
files = ["full_numpy_bitmap_crocodile.npy", "full_numpy_bitmap_lion.npy",
         "full_numpy_bitmap_lobster.npy", "full_numpy_bitmap_octopus.npy",
         "full_numpy_bitmap_panda.npy", "full_numpy_bitmap_swan.npy"]

max_images_per_class = 2000  # Limit to 1000 images per class

# For each class, we are giving them a corresponding label: 0 to 5
class_labels = {
    "full_numpy_bitmap_crocodile.npy": 0,
    "full_numpy_bitmap_lion.npy": 1,
    "full_numpy_bitmap_lobster.npy": 2,
    "full_numpy_bitmap_octopus.npy": 3,
    "full_numpy_bitmap_panda.npy": 4,
    "full_numpy_bitmap_swan.npy": 5
}

for filename in files:
    images = np.load(filename)
    print(f"Loaded {filename} with shape: {images.shape}")

    t_0 = time.perf_counter()
    count = 0

    # Loop through each image in the file
    for i in range(len(images)):
        # Only process the first 1000 images from each class
        if count >= max_images_per_class:
            break  # Stop once we've added 1000 images

        image = images[i]  # Provides (728,) array
        reshape = image.reshape(28, 28)  # Reshapes to (28, 28) numpy array
        image = Image.fromarray(reshape)

        # Assign the label based on the class of the file
        label = class_labels[filename]  # Get the label for the class

        data = {
            'image': image,  # The image tensor
            'label': label  # The corresponding label (class)
        }

        datasets2.append(data)
        count += 1

    t_1 = time.perf_counter()
    print(f"Successfully processed {filename} in {t_1 - t_0:.2f} seconds")


Loaded full_numpy_bitmap_crocodile.npy with shape: (127932, 784)
Successfully processed full_numpy_bitmap_crocodile.npy in 0.02 seconds
Loaded full_numpy_bitmap_lion.npy with shape: (120949, 784)
Successfully processed full_numpy_bitmap_lion.npy in 0.02 seconds
Loaded full_numpy_bitmap_lobster.npy with shape: (140175, 784)
Successfully processed full_numpy_bitmap_lobster.npy in 0.02 seconds
Loaded full_numpy_bitmap_octopus.npy with shape: (150152, 784)
Successfully processed full_numpy_bitmap_octopus.npy in 0.02 seconds
Loaded full_numpy_bitmap_panda.npy with shape: (113613, 784)
Successfully processed full_numpy_bitmap_panda.npy in 0.02 seconds
Loaded full_numpy_bitmap_swan.npy with shape: (152088, 784)
Successfully processed full_numpy_bitmap_swan.npy in 0.02 seconds


In [16]:
datasets2[11111]

{'image': <PIL.Image.Image image mode=L size=28x28>, 'label': 5}

In [17]:
def train_shuffled(num_img, batch_size=10, num_epoch=2):
    # Set up training parameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    total_loss = 0
    model.to(device)
    model.train()
    best_loss = 100

    # Define the label-to-description mapping for the text input
    label_to_description = {
        0: "a drawing of an octopus",
        1: "a drawing of a lion",
        2: "a drawing of a lobster",
        3: "a drawing of a panda",
        4: "a drawing of a swan",
        5: "a drawing of a crocodile"
    }

    for epoch in range(num_epoch):
        t0 = time.perf_counter()
        epoch_loss = 0

        # Separate training data into smaller batches
        for i in range(num_img // batch_size):
            # Select the current batch from the shuffled dataset
            start = i * batch_size
            end = (i + 1) * batch_size
            batch = datasets2[start:end]  # This is a list of batch size entries

            # Extract images and labels from the batch
            images = [item['image'] for item in batch]  # List of PIL images
            labels = [item['label'] for item in batch]  # List of numeric labels

            # Convert numeric labels to text descriptions (for the text input)
            text_labels = [label_to_description[label] for label in labels]

            t1 = time.perf_counter()

            # Convert images to tensors (assuming they need to be transformed first)
            images_tensor = processor(images=images, return_tensors="pt", padding=False).to(device)
            input_text = processor(text=text_labels, return_tensors="pt", padding=False).to(device)  # Process text labels as input

            # Combine image and text inputs
            input = {**images_tensor, **input_text}

            output = model(**input)

            # Get the logits for predictions on the image and text
            logits_per_image = output.logits_per_image
            logits_per_text = output.logits_per_text.squeeze()  # Make sure it's the correct shape

            # Convert labels to tensor and move to device (targets are class labels, kept as integers)
            targets = torch.tensor(labels).to(device)

            # Calculate loss
            loss_img = F.cross_entropy(logits_per_image, targets)
            loss_text = F.cross_entropy(logits_per_text, targets)
            loss = (loss_img + loss_text) / 2
            t2 = time.perf_counter()
            print(f"Finished batch {i + 1}/{num_img // batch_size} in {t2 - t1} seconds")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        total_loss += epoch_loss
        avg_loss = epoch_loss / (num_img // batch_size)
        t3 = time.perf_counter()

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model.state_dict(), 'clip.pt')

        print(f"Epoch {epoch+1}/{num_epoch} completed in {t3 - t0} seconds, Loss: {avg_loss:.4f}")


In [ ]:
train_shuffled(12000, batch_size=10, num_epoch=3)

Finished batch 1/1200 in 2.346813382000164 seconds
Finished batch 2/1200 in 0.5249338920000355 seconds
Finished batch 3/1200 in 0.5169751670000551 seconds
Finished batch 4/1200 in 0.5049896820000868 seconds
Finished batch 5/1200 in 0.518518155000038 seconds
Finished batch 6/1200 in 0.5233181250000598 seconds
Finished batch 7/1200 in 0.5240096189997985 seconds
Finished batch 8/1200 in 0.5221513620001588 seconds
Finished batch 9/1200 in 0.5221566499999426 seconds
Finished batch 10/1200 in 0.5312069530000372 seconds
Finished batch 11/1200 in 0.5331977190000998 seconds
Finished batch 12/1200 in 0.5264519459999519 seconds
Finished batch 13/1200 in 0.5362378239999543 seconds
Finished batch 14/1200 in 0.539415629000132 seconds
Finished batch 15/1200 in 0.5282753680000951 seconds
Finished batch 16/1200 in 0.5290913169999385 seconds
Finished batch 17/1200 in 0.5367414950001148 seconds
Finished batch 18/1200 in 0.5384062400000857 seconds
Finished batch 19/1200 in 0.5288515509998888 seconds
Finis